In [19]:
import networkx as nx
from gurobipy import *
from WFgenerator_tree import *
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum
import gurobipy as gp
from os import listdir
from os.path import isfile, join
import os
import time
import pandas as pd
from __future__ import print_function
import random as rand
import math, random
from parameter_set import *
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pickle

In [20]:
def MILP_solver(dag_batch):
   
    # Initialize the Gurobi model
    #dag_batch, wf_nodes, exec_nodes,num_exec, BigM = dag_setup()
    model = Model("Workflow_Scheduling_Problem")
    # Set Gurobi parameters
    model.setParam('MIPGap', 0.01)# Example
    model.setParam('TimeLimit', 720)  # 12 min limit
   
    # Define a large constant M for the big-M method in constraints
    M = 10000
    # Extract task and executor nodes from dag_batch
    wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
    exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]
    
# Decision Variables
    #Contionous Variables
    S = model.addVars([i for i in wf_nodes], name='S', lb=0, vtype='C')  # Start times
    F = model.addVars([i for i in wf_nodes], name='F', lb=0, vtype='C')  # Finish times
    d = model.addVars([i for i in wf_nodes],lb = 0,vtype='C', name = 'd')

# Binary Variables
    Psi = model.addVars([(i,j) for i in wf_nodes for j in wf_nodes], name='Psi', vtype='B')  # Precedence
    Phi = model.addVars([(j,y) for j in wf_nodes for y in exec_nodes], name='Phi', vtype='B')  # Task-executor assignments
    z = model.addVars([(i,j) for i in wf_nodes for j in wf_nodes], name='z', vtype='B')  # Warm-up cost indicator  
    

    # (1) Precedence Constraint (Sj≥Fi ∀i,j∈N, where χij=1)
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):               
                model.addConstr(S[j] >= F[i] +dag_batch.nodes[j]['WC'] * z[(i, j)], f"precedence_{i}_{j}")


    # (2) Finish Time Definition(Fj=Sj+tj ∀j∈N)
    for j in wf_nodes:
        model.addConstr(F[j] == S[j] + dag_batch.nodes[j]['dur'])

    # (3) Binary Relationship Constraint / Sequencing Constraint. 
    # (ψij+ψji=1∀i,j∈IY, i≠j,Y∈{1,2,…,w})
    for i in wf_nodes:
        for j in wf_nodes:
            if i != j:
                model.addConstr(Psi[i, j] + Psi[j, i] == 1)
                
    # (4) Scheduling Constraint(Si−Fj≥(ψij+ϕiy+ϕjy−3)M
    #∀i,j∈IY,i≠j,Y∈{1,2,…,w} Y####
    #∀i,j∈{wf_nodes},i≠j,y∈{exec_nodes}

    for i in wf_nodes:
        for j in wf_nodes:
            if i !=j:
                for y in exec_nodes:
                    #model.addConstr(S[i] - F[j] >= +dag_batch.nodes[j]['WC'] * z[i, j]+ (Psi[i, j] + Phi[i, y] + Phi[j, y] - 3) * M)
                    model.addConstr(S[i] - F[j] >= (Psi[i, j] + Phi[i, y] + Phi[j, y] - 3) * M)

    # (5) Task Assignment Constraint (∑_(y=1)^(f_Y)ϕjy=1 ∀j∈IY,Y∈{1,2,…,w})
    for j in wf_nodes:
        model.addConstr(sum(Phi[j, y] for y in exec_nodes) == 1)
                   

    # (6)Warm-up Cost Constraints 1
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)): 
                for y in exec_nodes:
                    model.addConstr(z[(i, j)] >= Phi[i, y] - Phi[j, y], f"warm_up_1_{i}_{j}_{y}")
                     
     # (7)Warm-up Cost Constraints 2                
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)): 
                for y in exec_nodes:
                    model.addConstr(z[(i, j)] >= Phi[j, y] - Phi[i, y], f"warm_up_2_{i}_{j}_{y}") 
                    
# #     # (2) Earliest Start Time Constraint (Sj≥ESj   ∀j∈N)
# #     # Start times of any operation should be after their corresponding ready times
#     for j in wf_nodes:
#         if dag_batch.nodes[j]['head'] == 1:
#             model.addConstr(S[j] >= dag_batch.nodes[j]['ready_time'])
            
# # Constraints
#     for i in wf_nodes:
#     # Get a list of available executors whose cpu, gpu and memory requirements are satisfied
#         avail_exec = [y for y in exec_nodes if dag_batch.nodes[i]['cpu'] <= dag_batch.nodes[y]['cpu'] and 
#                   dag_batch.nodes[i]['gpu'] <= dag_batch.nodes[y]['gpu'] and 
#                   dag_batch.nodes[i]['memory'] <= dag_batch.nodes[y]['memory']]
#         model.addConstr(gp.quicksum(Phi[(i,y)] for y in avail_exec) == 1)

# #     # (8) Due Date Constraint
#     for i in wf_nodes:
#         if dag_batch.nodes[i]['leaf'] == 1:
#             model.addConstr(d[i] >= F[i] - dag_batch.nodes[i]['deadline'])
            
    # Additional constraints involving Phi and Chi as per your model specifics
    # It can be satisfied by Gurobi's binary variable definition
    
# ------------------------------ END OF CONSTRAINTS---------------------------------------
    
    # Objective: Minimize the makespan including the total warm-up cost
    #total_warm_up_cost = sum(dag_batch.nodes[j]['C'] * z[i, j] for i in wf_nodes for j in wf_nodes if i != j)
    #model.setObjective(Z + total_warm_up_cost, GRB.MINIMIZE)
    
    obj = (gp.quicksum(dag_batch.nodes[j]['wt'] * (F[j]) for j in wf_nodes if dag_batch.nodes[j]['leaf'] == 1))

    # Solve the model
    model.setObjective(obj,GRB.MINIMIZE)
    model.optimize()
    print("Status of the model:", model.status)
    # Process and return results

    #return model 
#------------------------------------------SOLUTION-------------------------------------------------------

#          # Check if an optimal solution is found
#     if model.status == gp.GRB.OPTIMAL or model.status == gp.GRB.TIME_LIMIT:
#         F_ub = {j: F[i].X for i in wf_nodes}
#         S_ub = {j: S[j].X for j in wf_nodes}
#         return model, Psi, Phi, F_ub, S_ub
#     else:
#         print("No solution found!")
#         gap = None
#         return model, Psi, Phi, None, None
    
    
    # Check if an optimal solution is found
    if model.status == gp.GRB.OPTIMAL or model.status == gp.GRB.TIME_LIMIT:
        return model, Psi, Phi
    else:
        print("No solution found!")
        gap=None
        return model, Psi, Phi


LP only relaxing binary var

In [21]:
def LP_solver1(dag_batch, UB):
   
    # Initialize the Gurobi model
    #dag_batch, wf_nodes, exec_nodes,num_exec = dag_setupt()
    model = Model("Workflow_Scheduling_Problem")
    # Set Gurobi parameters
    model.setParam('MIPGap', 0.05)# Example
    #model.setParam('Presolve',0)
    model.setParam('TimeLimit', 300)  # 10 min limit
    # model.setParam('Method', ...)  # Choose method if necessary
    model.setParam('Cutoff', UB)
    
    # Define a large constant M for the big-M method in constraints
    
    # Extract task and executor nodes from dag_batch
    wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
    exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]
    
    #Compute bigM value
    # Let's analyze the practical range for Big M
    max_duration = max(dag_batch.nodes[j]['dur'] for j in wf_nodes)
    max_wc = max(dag_batch.nodes[j]['WC'] for j in wf_nodes)
    practical_bigM = max_duration + max_wc
    
    
    M=10000
    print("max_duration",max_duration)
    print("max_wc",max_wc)
    print("practical bigM",M)
    
# Decision Variables
    #Contionous Variables
    S = model.addVars([i for i in wf_nodes], name='S', lb=0, vtype='C')  # Start times
    F = model.addVars([i for i in wf_nodes], name='F', lb=0, vtype='C')  # Finish times
    #d = model.addVars([i for i in wf_nodes],lb = 0,vtype='C', name = 'd')

    #binary variable Relaxation ## LR #
    
    Psi = model.addVars(wf_nodes, wf_nodes, name='Psi', lb=0, ub=1, vtype='C')  # Precedence, now continuous
    Phi = model.addVars(wf_nodes, exec_nodes, name='Phi', lb=0, ub=1, vtype='C')  # Task-executor assignments, now continuous
    z = model.addVars(wf_nodes, wf_nodes, name='z', lb=0, ub=1, vtype='C')  # Warm-up cost indicator, now continuous


# Constraints

#     # (8) Due Date Constraint
#     for i in wf_nodes:
#         if dag_batch.nodes[i]['leaf'] == 1:
#             model.addConstr(d[i] >= F[i] - dag_batch.nodes[i]['deadline'])
            
    # (1) Precedence Constraint (Sj≥Fi ∀i,j∈N, where χij=1)
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):               
                model.addConstr(S[j] >= F[i] +dag_batch.nodes[j]['WC'] * z[(i, j)], f"precedence_{i}_{j}")


#     # (2) Earliest Start Time Constraint (Sj≥ESj   ∀j∈N)
#     # Start times of any operation should be after their corresponding ready times
#     for j in wf_nodes:
#         if dag_batch.nodes[j]['head'] == 1:
#             model.addConstr(S[j] >= dag_batch.nodes[j]['ready_time'])
 

    # (4) Finish Time Definition(Fj=Sj+tj ∀j∈N)
    for j in wf_nodes:
        model.addConstr(F[j] == S[j] + dag_batch.nodes[j]['dur'], f"Finish_Time_{j}")

    # (5) Binary Relationship Constraint / Sequencing Constraint. 
    # (ψij+ψji=1∀i,j∈IY, i≠j,Y∈{1,2,…,w})
    for i in wf_nodes:
        for j in wf_nodes:
            if i != j:
                model.addConstr(Psi[i, j] + Psi[j, i] == 1,f"Psi_Sequencing_{i}_{j}")
                
    # (6) Scheduling Constraint(Si−Fj≥(ψij+ϕiy+ϕjy−3)M
    ######∀i,j∈IY,i≠j,Y∈{1,2,…,w} Y####
    #∀i,j∈{wf_nodes},i≠j,y∈{exec_nodes}

    for i in wf_nodes:
        for j in wf_nodes:
            if i !=j:
                for y in exec_nodes:
                    model.addConstr(S[i] - F[j] >= (Psi[i, j] + Phi[i, y] + Phi[j, y] - 3) * M + dag_batch.nodes[j]['WC'] * z[(i, j)], f"Scheduling_{i}_{j}_{y}")
                    #model.addConstr(S[i] - F[j] >= +dag_batch.nodes[j]['WC'] * z[i, j]+ (Psi[i, j] + Phi[i, y] + Phi[j, y] - 3) * M)
    
    # (5) Task Assignment Constraint (∑_(y=1)^(f_Y)ϕjy=1 ∀j∈IY,Y∈{1,2,…,w})
    for j in wf_nodes:
        model.addConstr(sum(Phi[j, y] for y in exec_nodes) == 1, f"Task_Assignment_{j}")

    # (8)(9) Warm-up Cost Constraints 
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)): 
                for y in exec_nodes:
                    model.addConstr(z[(i, j)] >= Phi[i, y] - Phi[j, y], f"warm_up_1_{i}_{j}_{y}")
                    model.addConstr(z[(i, j)] >= Phi[j, y] - Phi[i, y], f"warm_up_2_{i}_{j}_{y}")     
  

#iff

# ------------------------------ END OF CONSTRAINTS---------------------------------------
    
    # Objective: Minimize the makespan including the total warm-up cost
    #total_warm_up_cost = sum(dag_batch.nodes[j]['C'] * z[i, j] for i in wf_nodes for j in wf_nodes if i != j)
    #model.setObjective(Z + total_warm_up_cost, GRB.MINIMIZE)
    
    obj = (gp.quicksum(dag_batch.nodes[j]['wt'] * (F[j]) for j in wf_nodes if dag_batch.nodes[j]['leaf'] == 1))

    # Solve the model
    model.setObjective(obj,GRB.MINIMIZE)
    model.optimize()
    print("Status of the model:", model.status)
    # Process and return results

    #return model 
#------------------------------------------SOLUTION-------------------------------------------------------

    # Check if an optimal solution is found
    if model.status == gp.GRB.OPTIMAL or model.status == gp.GRB.TIME_LIMIT:
        return model, Psi, Phi
    else:
        print("No solution found!")
        gap=None
        return model, Psi, Phi


Extracting dual from LP

In [22]:
def get_dual_values(model):
    """Extract non-zero dual values of the master problem's constraints and classify them into alpha, beta, gamma, theta, epsilon, and zeta."""
    duals = {}
    alpha = {}
    beta = {}
    gamma = {}
    theta = {}
    epsilon = {}
    zeta = {}

    model.update()  # Ensure the model is up-to-date
    for constr in model.getConstrs():
        try:
            dual_value = constr.getAttr('Pi')
            if dual_value != 0:
                constr_name = constr.ConstrName
                duals[constr_name] = dual_value

                if constr_name.startswith('precedence'):
                    parts = constr_name.split('_')
                    i = int(parts[1])
                    j = int(parts[2])
                    alpha[(i, j)] = dual_value
                elif constr_name.startswith('Finish_Time'):
                    j = int(constr_name.split('_')[2])
                    beta[j] = dual_value
                elif constr_name.startswith('warm_up_1'):
                    parts = constr_name.split('_')
                    i = int(parts[2])
                    j = int(parts[3])
                    y = int(parts[4])
                    gamma[(i, j, y)] = dual_value
                elif constr_name.startswith('warm_up_2'):
                    parts = constr_name.split('_')
                    i = int(parts[2])
                    j = int(parts[3])
                    y = int(parts[4])
                    theta[(i, j, y)] = dual_value
                elif constr_name.startswith('Scheduling'):
                    parts = constr_name.split('_')
                    i = int(parts[1])
                    j = int(parts[2])
                    epsilon[(i, j)] = dual_value
                elif constr_name.startswith('Task_Assignment'):
                    j = int(constr_name.split('_')[2])
                    zeta[j] = dual_value
        except AttributeError as e:
            print(f"Error retrieving dual value for constraint {constr.ConstrName}: {e}")
        except Exception as e:
            print(f"Unexpected error for constraint {constr.ConstrName}: {e}")

    return duals, alpha, beta, gamma, theta, epsilon, zeta

def normalize_subgradients(subgradients):
    norm = np.sqrt(sum(value**2 for value in subgradients.values())) + 1e-12
    return {key: value / norm for key, value in subgradients.items()}


Relaxing Constraint 1

In [23]:
def MILP_Relaxed_solver_Constr1(dag_batch, alpha):
    # Initialize the Gurobi model
    model = Model("Workflow_Scheduling_Problem")
    model.setParam('MIPGap', 0.05)
    model.setParam('TimeLimit', 180)  # 15 min limit

    # Set the cutoff value
    #model.setParam('Cutoff', cutoff_value)

    # Extract task and executor nodes from dag_batch
    wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
    exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]

    # Compute bigM value
    max_duration = max(dag_batch.nodes[j]['dur'] for j in wf_nodes)
    max_wc = max(dag_batch.nodes[j]['WC'] for j in wf_nodes)
    practical_bigM = max_duration + max_wc

    #M = practical_bigM
    M=10000
    # Decision Variables
    S = model.addVars(wf_nodes, name='S', lb=0, ub=300,vtype='C')  # Start times
    F = model.addVars(wf_nodes, name='F', lb=0, ub=300,vtype='C')  # Finish times
    Psi = model.addVars([(i, j) for i in wf_nodes for j in wf_nodes], name='Psi', vtype='B')  # Precedence
    Phi = model.addVars([(j, y) for j in wf_nodes for y in exec_nodes], name='Phi', vtype='B')  # Task-executor assignments
    z = model.addVars([(i, j) for i in wf_nodes for j in wf_nodes], name='z', vtype='B')  # Warm-up cost indicator  
    
#     #Ub Setting
#     max_F_ub = max(F_ub.values()) if F_ub else None
#     max_S_ub = max(S_ub.values()) if S_ub else None

#     if max_F_ub:
#         for j in wf_nodes:
#             F[j].ub = 1.1* max_F_ub
#     if max_S_ub:
#         for j in wf_nodes:
#             S[j].ub = 1.1* max_S_ub    
   
#     # Constraints
   
    # (4) Finish Time Definition (Fj = Sj + tj ∀j ∈ N)
    for j in wf_nodes:
        model.addConstr(F[j] == S[j] + dag_batch.nodes[j]['dur'], f"Finish_Time_{j}")

    # (5) Binary Relationship Constraint / Sequencing Constraint. 
    for i in wf_nodes:
        for j in wf_nodes:
            if i != j:
                model.addConstr(Psi[i, j] + Psi[j, i] == 1, f"Psi_Sequencing_{i}_{j}")

    #(6) Scheduling Constraint (Si - Fj ≥ (ψij + ϕiy + ϕjy - 3)M)
    for i in wf_nodes:
        for j in wf_nodes:
            if i != j:
                for y in exec_nodes:
                    model.addConstr(S[i] - F[j] >= (Psi[i, j] + Phi[i, y] + Phi[j, y] - 3) * M + dag_batch.nodes[j]['WC'] * z[(i, j)], f"Scheduling_{i}_{j}_{y}")

    # (5) Task Assignment Constraint (∑_(y=1)^(f_Y)ϕjy = 1 ∀j ∈ IY, Y ∈ {1, 2, …, w})
    for j in wf_nodes:
        model.addConstr(sum(Phi[j, y] for y in exec_nodes) == 1, f"Task_Assignment_{j}")

    # (8)(9) Warm-up Cost Constraints 
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):
                for y in exec_nodes:
                    model.addConstr(z[(i, j)] >= Phi[i, y] - Phi[j, y], f"warm_up_1_{i}_{j}_{y}")
                    model.addConstr(z[(i, j)] >= Phi[j, y] - Phi[i, y], f"warm_up_2_{i}_{j}_{y}")     

    # Objective Function
    obj = quicksum(dag_batch.nodes[j]['wt'] * F[j] for j in wf_nodes if dag_batch.nodes[j]['leaf'] == 1)
    
    if alpha: 
        print('alpha for obj', alpha) 
        penalty_term = quicksum(alpha.get((i, j), 0) * (F[i] - S[j] + dag_batch.nodes[j]['WC'] * z[(i, j)])
                       for i in wf_nodes for j in wf_nodes if (i, j) in alpha and j in list(dag_batch.successors(i)))
        obj += penalty_term

    model.setObjective(obj, GRB.MINIMIZE)

    var_values = {}
    for i in wf_nodes:
        for j in wf_nodes:
            var_values[(i, j)] = None
    model._var_values = var_values

    # Set callback to capture intermediate solutions
    model.optimize()
                                
    print("Status of the model:", model.status)
    subgradient = {}
    total_penalty = 0
    if model.status in [GRB.OPTIMAL, GRB.SUBOPTIMAL, GRB.TIME_LIMIT, GRB.INFEASIBLE, GRB.UNBOUNDED, GRB.INF_OR_UNBD] or model.SolCount > 0:
        # Collect variable values to compute subgradients
        for i in wf_nodes:
            for j in wf_nodes:
                if j in list(dag_batch.successors(i)):
                    try:
                        if (i, j) in alpha:
                            # Check if the variables have a solution value
                            if S[j].X is not None and F[i].X is not None and z[(i, j)].X is not None:
                                var_values[(i, j)] = {
                                    'S[j]': round(S[j].X, 2),
                                    'F[i]': round(F[i].X, 2),
                                    'z[i, j]': round(z[(i, j)].X, 2)
                                }
                                print("S,F,Z", S[j].X, F[i].X)
                                penalty_value = alpha[(i, j)] * (F[i].X - S[j].X + dag_batch.nodes[j]['WC'] * z[(i, j)].X)
                                subgradient[(i,j)]=(F[i].X - S[j].X + dag_batch.nodes[j]['WC'] * z[(i, j)].X)
                                total_penalty += penalty_value
                                print(f"Penalty for ({i}, {j}): {penalty_value}, alpha: {alpha[(i, j)]}, F[{i}]: {F[i].X}, S[{j}]: {S[j].X}, z[{i}, {j}]: {z[(i, j)].X}")
                    except (AttributeError, GurobiError) as e:
                        print(f"Error retrieving solution value for ({i}, {j}): {e}")
                        var_values[(i, j)] = {
                            'S[j]': None,
                            'F[i]': None,
                            'z[i, j]': None
                        }
    else:
        print("No feasible solution found.")

    print('total_penalty', total_penalty)

    return model, Psi, Phi, var_values, subgradient 

#adaptive_step_size = step_size / (iteration + 1)

def update_lagrange_multipliers_alpha(alpha, subgradients, step_size, iteration):
    updated_alpha = {}
    #adaptive_step_size = step_size / (iteration ** 0.5) if iteration > 0 else step_size  # Avoid division by zero
    adaptive_step_size = step_size / (iteration + 1)
    for key in alpha.keys():
        if key in subgradients and subgradients[key] is not None:
            updated_alpha[key] = max(0, alpha[key] + adaptive_step_size * subgradients[key])
        else:
            updated_alpha[key] = alpha[key]  # Keep the same value if subgradient is not valid

    return updated_alpha

In [24]:
def run_subgradient_method_Constr1(dag_batch, UB,LPB, MILP_Relaxed_solver_Constr1,
                                   alpha,normalize_subgradients, 
                                   update_lagrange_multipliers_alpha, num_iterations=100, tolerance=1e-4, patience=10):
    
    step_size = 1.0
    obj_values = []

    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):
                if (i, j) not in alpha:
                    alpha[(i, j)] = 0.1    

    alpha_updates = {key: [] for key in alpha.keys()}
    #best_obj_val = lp_model.ObjVal

    start_time = time.time()

    for iteration in range(num_iterations):
        model, Psi, Phi, var_values, subgradient = MILP_Relaxed_solver_Constr1(dag_batch, alpha)
        current_obj_val = model.ObjVal
        obj_values.append(current_obj_val)
        print('########################################################')
        print(f"Iteration {iteration + 1}")
        print("Current Objective Value:", current_obj_val)
        print("Dual values (alpha):", alpha)
        normalized_subgradients = normalize_subgradients(subgradient)
        print("Subgradients:", subgradient)
        
    # Early stopping check
        if len(obj_values) > 1:
            improvement = abs(obj_values[-2] - obj_values[-1])
            if improvement < tolerance:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    print("Early stopping due to negligible improvements.")
                    break
            else:
                no_improvement_count = 0
                
        alpha = update_lagrange_multipliers_alpha(alpha, normalized_subgradients, step_size, iteration)

        for key in alpha.keys():
            if key in alpha_updates:
                alpha_updates[key].append(alpha[key])

        print("Updated Lagrange multipliers (alpha):", alpha)

    end_time = time.time()
    total_time = end_time - start_time

    print('-------------------------------------------------------------------------------------------------------------------')
    print("Objective values over iterations:", obj_values)
    print("Maximum of minimum objective value:", max(obj_values))
    print("Total time taken for iterations:", total_time, "seconds")

    LRB = max(obj_values)
    gap = (UB - LRB) / UB * 100
    gap_LP = (UB - LPB) / UB * 100
    print("UB, LPB", UB, LPB)
    print('Gap between MILP and MILP_LR:', gap)
    print('Gap between MILP and LP:', gap_LP)

    return obj_values, total_time, gap, gap_LP, total_time_milp

In [25]:
def run_subgradient_method_Constr1_2(dag_batch, UB,LPB, MILP_Relaxed_solver_Constr1,
                                   alpha,normalize_subgradients, 
                                   update_lagrange_multipliers_alpha, num_iterations=100, tolerance=1e-4, patience=10):
    
    step_size = 1.0
    obj_values = []

    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):
                alpha[(i, j)] = 0.1    

    alpha_updates = {key: [] for key in alpha.keys()}
    #best_obj_val = lp_model.ObjVal

    start_time = time.time()

    for iteration in range(num_iterations):
        model, Psi, Phi, var_values, subgradient = MILP_Relaxed_solver_Constr1(dag_batch, alpha)
        current_obj_val = model.ObjVal
        obj_values.append(current_obj_val)
        print('########################################################')
        print(f"Iteration {iteration + 1}")
        print("Current Objective Value:", current_obj_val)
        print("Dual values (alpha):", alpha)
        normalized_subgradients = normalize_subgradients(subgradient)
        print("Subgradients:", subgradient)
        
    # Early stopping check
        if len(obj_values) > 1:
            improvement = abs(obj_values[-2] - obj_values[-1])
            if improvement < tolerance:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    print("Early stopping due to negligible improvements.")
                    break
            else:
                no_improvement_count = 0
                
        alpha = update_lagrange_multipliers_alpha(alpha, normalized_subgradients, step_size, iteration)

        for key in alpha.keys():
            if key in alpha_updates:
                alpha_updates[key].append(alpha[key])

        print("Updated Lagrange multipliers (alpha):", alpha)

    end_time = time.time()
    total_time = end_time - start_time

    print('-------------------------------------------------------------------------------------------------------------------')
    print("Objective values over iterations:", obj_values)
    print("Maximum of minimum objective value:", max(obj_values))
    print("Total time taken for iterations:", total_time, "seconds")

    LRB = max(obj_values)
    gap = (UB - LRB) / UB * 100
    gap_LP = (UB - LPB) / UB * 100
    print("UB, LPB", UB, LPB)
    print('Gap between MILP and MILP_LR:', gap)
    print('Gap between MILP and LP:', gap_LP)

    return obj_values, total_time, gap, gap_LP, total_time_milp

Relaxing Constraint 5

In [26]:

def MILP_Relaxed_solver_Constr5(dag_batch, epsilon):
    # Initialize the Gurobi model
    model = Model("Workflow_Scheduling_Problem")
    model.setParam('MIPGap', 0.03)
    model.setParam('TimeLimit', 180)  # 10 min limit
    #model.setParam('Cutoff', UB)
    # Extract task and executor nodes from dag_batch
    wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
    exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]

    # Compute bigM value
    max_duration = max(dag_batch.nodes[j]['dur'] for j in wf_nodes)
    max_wc = max(dag_batch.nodes[j]['WC'] for j in wf_nodes)
    practical_bigM = max_duration + max_wc

    M = practical_bigM
    print("M",M)
    # Decision Variables
    S = model.addVars(wf_nodes, name='S', lb=0, vtype='C')  # Start times
    F = model.addVars(wf_nodes, name='F', lb=0, vtype='C')  # Finish times
    Psi = model.addVars([(i,j) for i in wf_nodes for j in wf_nodes], name='Psi', vtype='B')  # Precedence
    Phi = model.addVars([(j,y) for j in wf_nodes for y in exec_nodes], name='Phi', vtype='B')  # Task-executor assignments
    z = model.addVars([(i,j) for i in wf_nodes for j in wf_nodes], name='z', vtype='B')  # Warm-up cost indicator  
    
    #UB
#     max_F_ub = max(F_ub.values()) if F_ub else None
#     max_S_ub = max(S_ub.values()) if S_ub else None

#     if max_F_ub:
#         for j in wf_nodes:
#             F[j].ub = 1.1* max_F_ub
#     if max_S_ub:
#         for j in wf_nodes:
#             S[j].ub = 1.1* max_S_ub    
            
    # Constraints
# (1) Precedence Constraint (Sj≥Fi ∀i,j∈N, where χij=1)
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):               
                model.addConstr(S[j] >= F[i] +dag_batch.nodes[j]['WC'] * z[(i, j)], f"precedence_{i}_{j}")


   
    # (2) Finish Time Definition (Fj = Sj + tj ∀j ∈ N)
    for j in wf_nodes:
        model.addConstr(F[j] == S[j] + dag_batch.nodes[j]['dur'], f"Finish_Time_{j}")

    # (3) Binary Relationship Constraint / Sequencing Constraint. 
    for i in wf_nodes:
        for j in wf_nodes:
            if i != j:
                model.addConstr(Psi[i, j] + Psi[j, i] == 1, f"Psi_Sequencing_{i}_{j}")

    # (5) Task Assignment Constraint (∑_(y=1)^(f_Y)ϕjy = 1 ∀j ∈ IY, Y ∈ {1, 2, …, w})
    for j in wf_nodes:
        model.addConstr(sum(Phi[j, y] for y in exec_nodes) == 1, f"Task_Assignment_{j}")

    # (6)(7) Warm-up Cost Constraints 
    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)):
                for y in exec_nodes:
                    model.addConstr(z[(i, j)] >= Phi[i, y] - Phi[j, y], f"warm_up_1_{i}_{j}_{y}")
                    model.addConstr(z[(i, j)] >= Phi[j, y] - Phi[i, y], f"warm_up_2_{i}_{j}_{y}")     

    # Objective Function
    obj = quicksum(dag_batch.nodes[j]['wt'] * F[j] for j in wf_nodes if dag_batch.nodes[j]['leaf'] == 1)
    
    if epsilon: 
        print('epsilon for obj', epsilon) 
        # Penalty term for relaxed scheduling constraints
        penalty_term_epsilon = quicksum(epsilon[(i, j, y)] * ((Psi[i, j] + Phi[i, y] + Phi[j, y] - 3) * M - S[i] + F[j] + dag_batch.nodes[j]['WC'] * z[(i, j)])
                       for i in wf_nodes for j in wf_nodes for y in exec_nodes if (i, j, y) in epsilon and j in list(dag_batch.successors(i)))
        obj += penalty_term_epsilon    
        
    model.setObjective(obj, GRB.MINIMIZE)                             
    model.optimize()
                                
                                
    print("Status of the model:", model.status)
                                
    subgradient = {}                            
    total_penalty=0
    var_values = {}
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.INFEASIBLE, GRB.UNBOUNDED, GRB.INF_OR_UNBD]:
        # Collect variable values to compute subgradients
        for i in wf_nodes:
            for j in wf_nodes:
                if j in list(dag_batch.successors(i)):
                    for y in exec_nodes:
                        try:
                            if (i, j, y) in epsilon:
                                var_values[(i, j, y)] = {
                                    'S[j]': round(S[j].X, 2),
                                    'F[i]': round(F[i].X, 2),
                                    'z[i, j]': round(z[(i, j)].X, 2),
                                    'Psi[i, j]': round(Psi[i, j].X, 2) if (i, j, y) in epsilon else None,
                                    'Phi[j, y]': round(Phi[j, y].X, 2) if (i, j, y) in epsilon else None,
                                    'Phi[i, y]': round(Phi[i, y].X, 2) if (i, j, y) in epsilon else None
                                }
                                
                                if (i, j, y) in epsilon:
                                    penalty_value_epsilon = epsilon[(i, j, y)] * ((Psi[i, j].X + Phi[i, y].X + Phi[j, y].X - 3) * M - S[i].X + F[j].X + dag_batch.nodes[j]['WC'] * z[(i, j)].X)
                                    subgradient[(i,j,y)]= ((Psi[i, j].X + Phi[i, y].X + Phi[j, y].X - 3) * M - S[i].X + F[j].X + dag_batch.nodes[j]['WC'] * z[(i, j)].X)
                                    total_penalty += penalty_value_epsilon
                                    print(f"Penalty for scheduling ({i}, {j}, {y}): {penalty_value_epsilon}, epsilon: {epsilon[(i, j, y)]}, S[{i}]: {S[i].X}, F[{j}]: {F[j].X}")
                        except (AttributeError, GurobiError) as e:
                            print(f"Error retrieving solution value for ({i}, {j}, {y}): {e}")
                            var_values[(i, j)] = {
                                'S[j]': None,
                                'F[i]': None,
                                'z[i, j]': None,
                                'Psi[i, j]': None,
                                'Phi[j, y]': None
                            }
        print('total_penalty', total_penalty)
        return model, Psi, Phi, var_values, subgradient
    else:
        print("No solution found!")
        return model, Psi, Phi, None


In [27]:
def update_lagrange_multipliers_epsilon(epsilon, subgradients, step_size, iteration):
    updated_epsilon = {}
    adaptive_step_size = step_size / (iteration + 1)
    for key in epsilon.keys():
        if key in subgradients:
            updated_epsilon[key] = max(0, epsilon[key] + adaptive_step_size * subgradients[key])  # Subtract subgradient
        else:
            updated_epsilon[key] = epsilon[key]
    return updated_epsilon

In [28]:
def run_subgradient_method_Constr5(dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, 
                                   epsilon, normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=100, tolerance=1e-4, patience=10):   
    
    step_size = 1.0
    obj_values = []

    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)) and i != j:
                for y in exec_nodes:
                    if (i, j, y) not in epsilon:
                        epsilon[(i, j, y)] = 0.1

    epsilon_updates = {key: [] for key in epsilon.keys()}  # Initialize a dictionary to track epsilon updates

    # Measure the start time
    start_time = time.time()

    for iteration in range(num_iterations):
        model, Psi, Phi, var_values, subgradient = MILP_Relaxed_solver_Constr5(dag_batch, epsilon)
        if var_values is None:
            break
        current_obj_val = model.ObjVal
        obj_values.append(model.ObjVal)
        print('########################################################')
        print(f"Iteration {iteration + 1}")
        print("Current Objective Value:", current_obj_val)
        print("Dual values (epsilon):", epsilon)
        print("Subgradients:", subgradient)
        normalized_subgradients = normalize_subgradients(subgradient)
        if len(obj_values) > 1:
            improvement = abs(obj_values[-2] - obj_values[-1])
            if improvement < tolerance:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    print("Early stopping due to negligible improvements.")
                    break
            else:
                no_improvement_count = 0     

        epsilon = update_lagrange_multipliers_epsilon(epsilon, normalized_subgradients, step_size, iteration)

        # Track the updates of epsilon values
        for key in epsilon.keys():
            if key in epsilon_updates:
                epsilon_updates[key].append(epsilon[key])

        # Print updated epsilon values
        print("Updated Lagrange multipliers (epsilon):", epsilon)

    # Measure the end time
    end_time = time.time()

    # Calculate the total duration
    total_time = end_time - start_time
    print('-------------------------------------------------------------------------------------------------------------------')
    print("Total time", total_time)
    print("Objective values over iterations:", obj_values)
    LRB = max(obj_values)
    print("Maximum of minimum objective value:", LRB)
    gap_LP=(UB-LPB)/UB*100
    gap= (UB-LRB)/UB *100
    print("Computation time", total_time)
    print('gap btw MILP and MILP_LR',gap)
    print('gap btw MILP and LP',gap_LP)

    return obj_values, total_time, gap, gap_LP, total_time_milp





In [29]:
def run_subgradient_method_Constr5_2(dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, 
                                   epsilon, normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=100, tolerance=1e-4, patience=10):   
    
    step_size = 1.0
    obj_values = []

    for i in wf_nodes:
        for j in wf_nodes:
            if j in list(dag_batch.successors(i)) and i != j:
                for y in exec_nodes:
                    epsilon[(i, j, y)] = 0.1

    epsilon_updates = {key: [] for key in epsilon.keys()}  # Initialize a dictionary to track epsilon updates

    # Measure the start time
    start_time = time.time()

    for iteration in range(num_iterations):
        model, Psi, Phi, var_values, subgradient = MILP_Relaxed_solver_Constr5(dag_batch, epsilon)
        if var_values is None:
            break
        current_obj_val = model.ObjVal
        obj_values.append(model.ObjVal)
        print('########################################################')
        print(f"Iteration {iteration + 1}")
        print("Current Objective Value:", current_obj_val)
        print("Dual values (epsilon):", epsilon)
        print("Subgradients:", subgradient)
        normalized_subgradients = normalize_subgradients(subgradient)
        if len(obj_values) > 1:
            improvement = abs(obj_values[-2] - obj_values[-1])
            if improvement < tolerance:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    print("Early stopping due to negligible improvements.")
                    break
            else:
                no_improvement_count = 0     

        epsilon = update_lagrange_multipliers_epsilon(epsilon, normalized_subgradients, step_size, iteration)

        # Track the updates of epsilon values
        for key in epsilon.keys():
            if key in epsilon_updates:
                epsilon_updates[key].append(epsilon[key])

        # Print updated epsilon values
        print("Updated Lagrange multipliers (epsilon):", epsilon)

    # Measure the end time
    end_time = time.time()

    # Calculate the total duration
    total_time = end_time - start_time
    print('-------------------------------------------------------------------------------------------------------------------')
    print("Total time", total_time)
    print("Objective values over iterations:", obj_values)
    LRB = max(obj_values)
    print("Maximum of minimum objective value:", LRB)
    gap_LP=(UB-LPB)/UB*100
    gap= (UB-LRB)/UB *100
    print("Computation time", total_time)
    print('gap btw MILP and MILP_LR',gap)
    print('gap btw MILP and LP',gap_LP)

    return obj_values, total_time, gap, gap_LP, total_time_milp





Iteration Loop

In [30]:
# 3333 _ 0.03 180 comparing 1,5,6

In [31]:
# Define the directory where the pickle files are saved
directory = "C:/Users/yoonhwankang/Cloud native scheduling Problem/medium size data/batch 1"

# Get a list of all pickle files in the directory
pkl_files = [f for f in os.listdir(directory) if f.endswith('.pkl')]

# Number of experiments to run
num_experiments = len(pkl_files)  # Ensure we don't try to run more experiments than we have files
num_experiments

5

In [ ]:
# Initialize lists to store results
all_obj_values_1 = []
all_obj_values_1_2 = []
all_obj_values_5 = []
all_obj_values_5_2 = []
all_obj_values_6 = []

all_times_1 = [] 
all_times_1_2 = [] 
all_times_5 = []
all_times_5_2 = []
all_times_6 = []

all_times_milp_1 = [] 
all_times_milp_1_2 = []
all_times_milp_5 = []
all_times_milp_5_2 = []
all_times_milp_6 = []

best_obj_values_1 = []
best_obj_values_1_2 = []
best_obj_values_5 = []
best_obj_values_5_2 = []
best_obj_values_6 = []

gap_records_1 = []
gap_records_1_2 = []
gap_records_5 = []
gap_records_5_2 = []
gap_records_6 = []

gap_LP_record_1 = []
gap_LP_record_1_2 = []
gap_LP_record_5 = []
gap_LP_record_5_2 = []
gap_LP_record_6 = []

nodes_record = []
exec_record = []
milp_record = []
lp_record = []
#num_experiments = 1

# Loop through the files and run the experiments
for i in range(num_experiments):
    file_path = os.path.join(directory, pkl_files[i])
    
    try:
        print(f"Running experiment {i + 1}/{num_experiments} with file {pkl_files[i]}")
        
        # Load the graph using pickle
        with open(file_path, 'rb') as f:
            dag_batch = pickle.load(f)
        
        wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
        exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]
        num_nodes = len(wf_nodes)
        print("num nodes", num_nodes)
        num_exec=len(exec_nodes)
        
        # MILP
        start_time_milp = time.time()
        model_MILP, Psi, Phi = MILP_solver(dag_batch)
        # Check if F_ub or S_ub is None
#         if F_ub is None or S_ub is None:
#             print(f"F_ub or S_ub is None. Skipping experiment {i + 1}.")
#             continue  # Skip this experiment
        end_time_milp = time.time()
        total_time_milp = end_time_milp - start_time_milp
        print('MILP Obj', model_MILP.ObjVal)
        print('MILP Time', total_time_milp)
        UB = model_MILP.ObjVal

        # Solve initial LP
        lp_model, Psi, Phi = LP_solver1(dag_batch, UB)
        LPB = lp_model.ObjVal
        
        # Check if LP objective exceeds MILP objective
        if LPB > UB:
            print(f"LP objective ({LPB}) exceeds MILP objective ({UB}). Stopping experiment.")
            continue  # Skip this experiment
        print('LP Obj', LPB)
        duals, alpha, beta, gamma, theta, epsilon, zeta = get_dual_values(lp_model)
        print("duals", duals)
        
#         # Running subgradient method for constraint 5
#         obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr5(
#             dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, epsilon,
#             normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=50, tolerance=1e-4, patience=10
#         )
        
#         # Append results for constraint 5
#         all_obj_values_5.append(obj_values)
#         all_times_5.append(total_time)
#         all_times_milp_5.append(total_time_milp)
#         gap_records_5.append(gap)
#         gap_LP_record_5.append(gap_LP)
        
#         # Running subgradient method for constraint 5_2
#         obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr5_2(
#             dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, epsilon,
#             normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=50, tolerance=1e-4, patience=10
#         )
        
#         # Append results for constraint 5_2
#         all_obj_values_5_2.append(obj_values)
#         all_times_5_2.append(total_time)
#         all_times_milp_5_2.append(total_time_milp)
#         gap_records_5_2.append(gap)
#         gap_LP_record_5_2.append(gap_LP)
        
        print("-------------constr1---------------------")
        # Running subgradient method for constraint 1
        obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr1(
            dag_batch, UB, LPB, MILP_Relaxed_solver_Constr1, alpha,
            normalize_subgradients, update_lagrange_multipliers_alpha, num_iterations=30, tolerance=1e-4, patience=10
        )
        # Append results for constraint 1
        all_obj_values_1.append(obj_values)
        all_times_1.append(total_time)
        all_times_milp_1.append(total_time_milp)
        gap_records_1.append(gap)
        gap_LP_record_1.append(gap_LP)
        
        # Running subgradient method for constraint 1_2
        obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr1_2(
            dag_batch, UB, LPB, MILP_Relaxed_solver_Constr1, alpha,
            normalize_subgradients, update_lagrange_multipliers_alpha, num_iterations=30, tolerance=1e-4, patience=10
        )
        # Append results for constraint 1_2
        all_obj_values_1_2.append(obj_values)
        all_times_1_2.append(total_time)
        all_times_milp_1_2.append(total_time_milp)
        gap_records_1_2.append(gap)
        gap_LP_record_1_2.append(gap_LP)
        
        nodes_record.append(num_nodes)
        exec_record.append(num_exec)
        milp_record.append(UB)
        lp_record.append(LPB)
        
    except Exception as e:
        print(f"Experiment {i + 1} failed with error: {e}")
        continue  # Continue with the next experiment if there is an error

# Print the completion message
print("All experiments completed.")


Running experiment 1/5 with file dag_batch_10_125.89583778381348.pkl
num nodes 32
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 720
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4375 rows, 2240 columns and 18043 nonzeros
Model fingerprint: 0x66962d4e
Variable types: 96 continuous, 2144 integer (2144 binary)
Coefficient statistics:
  Matrix range     [8e-02, 1e+04]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 3e+04]
Presolve removed 1084 rows and 1577 columns
Presolve time: 0.05s
Presolved: 3291 rows, 663 columns, 15815 nonzeros
Variable types: 32 continuous, 631 integer (631 binary)
Found heuristic solution: objective 2398.8050000

Root relaxation: objective 1.015478e+03, 208 iterations, 0.01 seconds (0.00 work uni

In [17]:
# Create DataFrame from the data lists
data = {
    'all_obj_values_1': all_obj_values_1,
    'all_obj_values_1_2': all_obj_values_1_2,
    'all_times_1': all_times_1,
    'all_times_1_2': all_times_1_2,
    'all_times_milp_1': all_times_milp_1,
    'all_times_milp_1_2': all_times_milp_1_2,
    'gap_records_1': gap_records_1,
    'gap_records_1_2': gap_records_1_2,
    'gap_LP_record_1': gap_LP_record_1,
    'gap_LP_record_1_2': gap_LP_record_1_2,
    'all_obj_values_5': all_obj_values_5,
    'all_obj_values_5_2': all_obj_values_5_2,
    'all_times_5': all_times_5,
    'all_times_5_2': all_times_5_2,
    'all_times_milp_5': all_times_milp_5,
    'all_times_milp_5_2': all_times_milp_5_2,
    'gap_records_5': gap_records_5,
    'gap_records_5_2': gap_records_5_2,
    'gap_LP_record_5': gap_LP_record_5,
    'gap_LP_record_5_2': gap_LP_record_5_2,
    'nodes_record': nodes_record,
    'exec_record': exec_record,
    'milp_record': milp_record,
    'lp_record': lp_record
}

df = pd.DataFrame(data)
df



,all_obj_values_1,all_obj_values_1_2,all_times_1,all_times_1_2,all_times_milp_1,all_times_milp_1_2,gap_records_1,gap_records_1_2,gap_LP_record_1,gap_LP_record_1_2,...,all_times_milp_5,all_times_milp_5_2,gap_records_5,gap_records_5_2,gap_LP_record_5,gap_LP_record_5_2,nodes_record,exec_record,milp_record,lp_record


In [ ]:
# Save the DataFrame to a CSV file
csv_file_path = "C:/Users/yoonhwankang/Cloud_native_scheduling_Problem/LR_comparison_with_medium_size_inital1.csv "
df.to_csv(csv_file_path, index=False)

print(f"Results saved to {csv_file_path}")

In [ ]:
# Define the directory where the pickle files are saved
directory = "C:/Users/yoonhwankang/Cloud native scheduling Problem/small size data"

# Get a list of all pickle files in the directory
pkl_files = [f for f in os.listdir(directory) if f.endswith('.pkl')]

# Number of experiments to run
num_experiments = len(pkl_files)  # Ensure we don't try to run more experiments than we have files
num_experiments

In [ ]:
# Initialize lists to store results
all_obj_values_1 = []
all_obj_values_1_2 = []
all_obj_values_5 = []
all_obj_values_5_2 = []
all_obj_values_6 = []

all_times_1 = [] 
all_times_1_2 = [] 
all_times_5 = []
all_times_5_2 = []
all_times_6 = []

all_times_milp_1 = [] 
all_times_milp_1_2 = []
all_times_milp_5 = []
all_times_milp_5_2 = []
all_times_milp_6 = []

best_obj_values_1 = []
best_obj_values_1_2 = []
best_obj_values_5 = []
best_obj_values_5_2 = []
best_obj_values_6 = []

gap_records_1 = []
gap_records_1_2 = []
gap_records_5 = []
gap_records_5_2 = []
gap_records_6 = []

gap_LP_record_1 = []
gap_LP_record_1_2 = []
gap_LP_record_5 = []
gap_LP_record_5_2 = []
gap_LP_record_6 = []

nodes_record = []
exec_record = []
milp_record = []
lp_record = []
#num_experiments = 1

# Loop through the files and run the experiments
for i in range(num_experiments):
    file_path = os.path.join(directory, pkl_files[i])
    
    try:
        print(f"Running experiment {i + 1}/{num_experiments} with file {pkl_files[i]}")
        
        # Load the graph using pickle
        with open(file_path, 'rb') as f:
            dag_batch = pickle.load(f)
        
        wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
        exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]
        num_nodes = len(wf_nodes)
        print("num nodes", num_nodes)
        num_exec=len(exec_nodes)
        
        # MILP
        start_time_milp = time.time()
        model_MILP, Psi, Phi = MILP_solver(dag_batch)
        end_time_milp = time.time()
        total_time_milp = end_time_milp - start_time_milp
        print('MILP Obj', model_MILP.ObjVal)
        print('MILP Time', total_time_milp)
        UB = model_MILP.ObjVal

        # Solve initial LP
        lp_model, Psi, Phi = LP_solver1(dag_batch, UB)
        LPB = lp_model.ObjVal
        
        # Check if LP objective exceeds MILP objective
        if LPB > UB:
            print(f"LP objective ({LPB}) exceeds MILP objective ({UB}). Stopping experiment.")
            continue  # Skip this experiment
        print('LP Obj', LPB)
        duals, alpha, beta, gamma, theta, epsilon, zeta = get_dual_values(lp_model)
        print("duals", duals)
        
        # Running subgradient method for constraint 5
        obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr5(
            dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, epsilon,
            normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=50, tolerance=1e-4, patience=10
        )
        
        # Append results for constraint 5
        all_obj_values_5.append(obj_values)
        all_times_5.append(total_time)
        all_times_milp_5.append(total_time_milp)
        gap_records_5.append(gap)
        gap_LP_record_5.append(gap_LP)
        
        # Running subgradient method for constraint 5_2
        obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr5_2(
            dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, epsilon,
            normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=50, tolerance=1e-4, patience=10
        )
        
        # Append results for constraint 5_2
        all_obj_values_5_2.append(obj_values)
        all_times_5_2.append(total_time)
        all_times_milp_5_2.append(total_time_milp)
        gap_records_5_2.append(gap)
        gap_LP_record_5_2.append(gap_LP)
        
        print("-------------constr1---------------------")
        # Running subgradient method for constraint 1
        obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr1(
            dag_batch, UB, LPB, MILP_Relaxed_solver_Constr1, alpha,
            normalize_subgradients, update_lagrange_multipliers_alpha, num_iterations=30, tolerance=1e-4, patience=10
        )
        # Append results for constraint 1
        all_obj_values_1.append(obj_values)
        all_times_1.append(total_time)
        all_times_milp_1.append(total_time_milp)
        gap_records_1.append(gap)
        gap_LP_record_1.append(gap_LP)
        
        # Running subgradient method for constraint 1_2
        obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr1_2(
            dag_batch, UB, LPB, MILP_Relaxed_solver_Constr1, alpha,
            normalize_subgradients, update_lagrange_multipliers_alpha, num_iterations=30, tolerance=1e-4, patience=10
        )
        # Append results for constraint 1_2
        all_obj_values_1_2.append(obj_values)
        all_times_1_2.append(total_time)
        all_times_milp_1_2.append(total_time_milp)
        gap_records_1_2.append(gap)
        gap_LP_record_1_2.append(gap_LP)
        
        nodes_record.append(num_nodes)
        exec_record.append(num_exec)
        milp_record.append(UB)
        lp_record.append(LPB)
        
    except Exception as e:
        print(f"Experiment {i + 1} failed with error: {e}")
        continue  # Continue with the next experiment if there is an error

# Print the completion message
print("All experiments completed.")


In [ ]:
# Create DataFrame from the data lists
data = {
    'all_obj_values_1': all_obj_values_1,
    'all_obj_values_1_2': all_obj_values_1_2,
    'all_times_1': all_times_1,
    'all_times_1_2': all_times_1_2,
    'all_times_milp_1': all_times_milp_1,
    'all_times_milp_1_2': all_times_milp_1_2,
    'gap_records_1': gap_records_1,
    'gap_records_1_2': gap_records_1_2,
    'gap_LP_record_1': gap_LP_record_1,
    'gap_LP_record_1_2': gap_LP_record_1_2,
    'all_obj_values_5': all_obj_values_5,
    'all_obj_values_5_2': all_obj_values_5_2,
    'all_times_5': all_times_5,
    'all_times_5_2': all_times_5_2,
    'all_times_milp_5': all_times_milp_5,
    'all_times_milp_5_2': all_times_milp_5_2,
    'gap_records_5': gap_records_5,
    'gap_records_5_2': gap_records_5_2,
    'gap_LP_record_5': gap_LP_record_5,
    'gap_LP_record_5_2': gap_LP_record_5_2,
    'nodes_record': nodes_record,
    'exec_record': exec_record,
    'milp_record': milp_record,
    'lp_record': lp_record
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = "C:/Users/yoonhwankang/Cloud_native_scheduling_Problem/LR_comparison_with_medium_size_inital2.csv"
df.to_csv(csv_file_path, index=False)

print(f"Results saved to {csv_file_path}")


In [ ]:
# # Initialize lists to store results
# all_obj_values_1 = []
# all_obj_values_1_2 = []
# all_obj_values_5 = []
# all_obj_values_5_2 = []
# all_obj_values_6 = []

# all_times_1_2 = [] 
# all_times_5 = []
# all_times_5_2 = []
# all_times_6 = []

# all_times_milp_1 = [] 
# all_times_milp_1_2 = []
# all_times_milp_5 = []
# all_times_milp_5_2 = []
# all_times_milp_6 = []

# best_obj_values_1 = []
# best_obj_values_1_2 = []
# best_obj_values_5 = []
# best_obj_values_5_2 = []
# best_obj_values_6 = []

# gap_records_1 = []
# gap_records_1_2 = []
# gap_records_5 = []
# gap_records_5_2 = []
# gap_records_6 = []

# gap_LP_record_1 = []
# gap_LP_record_1_2 = []
# gap_LP_record_5 = []
# gap_LP_record_5_2 = []
# gap_LP_record_6 = []

# nodes_record = []
# exec_record = []
# milp_record = []
# lp_record = []
# num_experiments = 3

# # Loop through the files and run the experiments
# for i in range(num_experiments):
#     file_path = os.path.join(directory, pkl_files[i])
    
#     try:
#         print(f"Running experiment {i + 1}/{num_experiments} with file {pkl_files[i]}")
        
#         # Load the graph using pickle
#         with open(file_path, 'rb') as f:
#             dag_batch = pickle.load(f)
        
#         wf_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 1]
#         exec_nodes = [n for n in dag_batch.nodes if dag_batch.nodes[n]['wf'] == 0]
#         num_nodes = len(wf_nodes)
#         print("num nodes", num_nodes)
        
#         # MILP
#         start_time_milp = time.time()
#         model_MILP, Psi, Phi = MILP_solver(dag_batch)
#         end_time_milp = time.time()
#         total_time_milp = end_time_milp - start_time_milp
#         print('MILP Obj', model_MILP.ObjVal)
#         print('MILP Time', total_time_milp)
#         UB = model_MILP.ObjVal

#         # Solve initial LP
#         lp_model, Psi, Phi = LP_solver1(dag_batch, UB)
#         LPB = lp_model.ObjVal
        
#         # Check if LP objective exceeds MILP objective
#         if LPB > UB:
#             print(f"LP objective ({LPB}) exceeds MILP objective ({UB}). Stopping experiment.")
#             continue  # Skip this experiment
#         print('LP Obj', LPB)
#         duals, alpha, beta, gamma, theta, epsilon, zeta = get_dual_values(lp_model)
#         print("duals", duals)
        
#         # Running subgradient method for constraint 5
#         obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr5(
#             dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5, epsilon,
#             normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=50, tolerance=1e-4, patience=10
#         )
        
#         # Append results for constraint 5
#         all_obj_values_5.append(obj_values)
#         all_times_5.append(total_time)
#         all_times_milp_5.append(total_time_milp)
#         gap_records_5.append(gap)
#         gap_LP_record_5.append(gap_LP)
        
#         # Running subgradient method for constraint 5_2
#         obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr5_2(
#             dag_batch, UB, LPB, MILP_Relaxed_solver_Constr5_2, epsilon,
#             normalize_subgradients, update_lagrange_multipliers_epsilon, num_iterations=50, tolerance=1e-4, patience=10
#         )
        
#         # Append results for constraint 5_2
#         all_obj_values_5_2.append(obj_values)
#         all_times_5_2.append(total_time)
#         all_times_milp_5_2.append(total_time_milp)
#         gap_records_5_2.append(gap)
#         gap_LP_record_5_2.append(gap_LP)
        
#         print("-------------constr1---------------------")
#         # Running subgradient method for constraint 1
#         obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr1(
#             dag_batch, UB, LPB, MILP_Relaxed_solver_Constr1, alpha,
#             normalize_subgradients, update_lagrange_multipliers_alpha, num_iterations=50, tolerance=1e-4, patience=10
#         )
#         # Append results for constraint 1
#         all_obj_values_1.append(obj_values)
#         all_times_1.append(total_time)
#         all_times_milp_1.append(total_time_milp)
#         gap_records_1.append(gap)
#         gap_LP_record_1.append(gap_LP)
        
#         # Running subgradient method for constraint 1_2
#         obj_values, total_time, gap, gap_LP, total_time_milp = run_subgradient_method_Constr1_2(
#             dag_batch, UB, LPB, MILP_Relaxed_solver_Constr1_2, alpha,
#             normalize_subgradients, update_lagrange_multipliers_alpha, num_iterations=50, tolerance=1e-4, patience=10
#         )
#         # Append results for constraint 1_2
#         all_obj_values_1_2.append(obj_values)
#         all_times_1_2.append(total_time)
#         all_times_milp_1_2.append(total_time_milp)
#         gap_records_1_2.append(gap)
#         gap_LP_record_1_2.append(gap_LP)
        
#         nodes_record.append(num_nodes)
#         exec_record.append(num_exec)
#         milp_record.append(UB)
#         lp_record.append(LPB)
        
#     except Exception as e:
#         print(f"Experiment {i + 1} failed with error: {e}")
#         continue  # Continue with the next experiment if there is an error

# # Print the completion message
# print("All experiments completed.")
